In [4]:
library("readxl")
library(rvest) # scrap web pages
library(reticulate)  # R & python interchange
library(tidyverse)

Loading required package: xml2

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.1     ✔ purrr   0.3.4
✔ tibble  3.0.1     ✔ dplyr   1.0.0
✔ tidyr   1.1.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter()         masks stats::filter()
✖ readr::guess_encoding() masks rvest::guess_encoding()
✖ dplyr::lag()            masks stats::lag()
✖ purrr::pluck()          masks rvest::pluck()



## Import BIB data
- https://bibliometrix.org/documents/bibliometrix_Report.html
- Place your bibtex files in bibs folder

In [20]:
# Reading Bibtex
# assuming that you are in the same directory as this snippet
library(bibliometrix)
bibs <- list.files("bibs/", pattern = "bib$", recursive = T, full.names = T)
bibs # visualize the files, index start at 1

[1] "bibs//cochrane.bib"  "bibs//savedrecs.bib" "bibs//scopus.bib"

In [ ]:
M <- convert2df(file=bibs[1], dbsource = "scopus", format = "bibtex")

## Import xlsx data
File extracted from StArt Tool

In [ ]:
my_data <- read_excel("arquivos.xlsx")
sub_data <- my_data[my_data$`Status/Selection` == "ACCEPTED",]
names(sub_data)

In [12]:
head(D)

[1] ""                                                                                                              
[2] "@ARTICLE{Mylonas201943,"                                                                                       
[3] "author={Mylonas, G. and Amaxilatis, D. and Pocero, L. and Markelis, I. and Hofstaetter, J. and Koulouris, P.},"
[4] "title={An educational IoT lab kit and tools for energy awareness in European schools},"                        
[5] "journal={International Journal of Child-Computer Interaction},"                                                
[6] "year={2019},"

In [11]:
getwd()

[1] "/home/laccan/testesRNotebooks/rnatlp"